# SMB Create Remote File

## Metadata


|                   |    |
|:------------------|:---|
| collaborators     | ['Roberto Rodriguez @Cyb3rWard0g', 'Jose Rodriguez @Cyb3rPandaH'] |
| creation date     | 2020/10/12 |
| modification date | 2020/10/12 |
| playbook related  | [] |

## Hypothesis
Adversaries might be creating a file remotely via the Server Message Block (SMB) Protocol.

## Technical Context
Client systems use the Common Internet File System (CIFS) Protocol to request file and print services from server systems over a network. CIFS is a stateful protocol, in which clients establish a session with a server and use that session to make a variety of requests to access files, printers, and inter-process communication (IPC) mechanisms, such as named pipes.
The extended CIFS Protocol is known as the Server Message Block (SMB). The SMB2 CREATE Request packet is sent by a client to request either creation of or access to a file. In case of a named pipe or printer, the server MUST create a new file.

## Offensive Tradecraft
Adversaries leverage SMB to copy files over the network to either execute code remotely or exfiltrate data.

## Mordor Test Data


|           |           |
|:----------|:----------|
| metadata  | https://mordordatasets.com/notebooks/small/windows/08_lateral_movement/SDWIN-200806015757.html        |
| link      | [https://raw.githubusercontent.com/OTRF/mordor/master/datasets/small/windows/lateral_movement/host/covenant_copy_smb_CreateRequest.zip](https://raw.githubusercontent.com/OTRF/mordor/master/datasets/small/windows/lateral_movement/host/covenant_copy_smb_CreateRequest.zip)  |

## Analytics

### Initialize Analytics Engine

In [ ]:
from openhunt.mordorutils import *
spark = get_spark()

### Download & Process Mordor Dataset

In [ ]:
mordor_file = "https://raw.githubusercontent.com/OTRF/mordor/master/datasets/small/windows/lateral_movement/host/covenant_copy_smb_CreateRequest.zip"
registerMordorSQLTable(spark, mordor_file, "mordorTable")

### Analytic I
Look for non-system accounts SMB connecting (Tree Connect) to a file share that is not IPC$.


| Data source | Event Provider | Relationship | Event |
|:------------|:---------------|--------------|-------|
| File | Microsoft-Windows-Security-Auditing | User accessed file share | 5140 |

In [ ]:
df = spark.sql(
'''
SELECT `@timestamp`, Hostname, ShareName, SubjectUserName, SubjectLogonId,  AccessMask
FROM mordorTable
WHERE LOWER(Channel) = 'security'
    AND (EventID = 5140)
    AND NOT ShareName LIKE '%IPC$'
    AND NOT SubjectUserName LIKE '%$'
'''
)
df.show(10,False)

### Analytic II
Look for non-system accounts SMB connecting (Tree Connect) to an IPC$ Share and administrative shares (i.e C$) with the same logon session ID.


| Data source | Event Provider | Relationship | Event |
|:------------|:---------------|--------------|-------|
| File | Microsoft-Windows-Security-Auditing | User accessed file share | 5140 |

In [ ]:
df = spark.sql(
'''
SELECT `@timestamp`, Hostname, ShareName, SubjectUserName, b.SubjectLogonId, IpAddress, IpPort
FROM mordorTable b
INNER JOIN (
    SELECT SubjectLogonId
    FROM mordorTable
    WHERE LOWER(Channel) = "security"
        AND EventID = 5140
        AND ShareName LIKE '%IPC$'
        AND NOT SubjectUserName LIKE '%$'
    ) a
ON b.SubjectLogonId = a.SubjectLogonId
WHERE LOWER(b.Channel) = 'security'
    AND b.EventID = 5140
    AND b.ShareName LIKE '%C$'
    AND NOT SubjectUserName LIKE '%$'
'''
)
df.show(10,False)

### Analytic III
Look for non-system accounts SMB accessing a file with write (0x2) access mask via administrative share (i.e C$).


| Data source | Event Provider | Relationship | Event |
|:------------|:---------------|--------------|-------|
| File | Microsoft-Windows-Security-Auditing | User accessed File | 5145 |

In [ ]:
df = spark.sql(
'''
SELECT `@timestamp`, Hostname, ShareName, SubjectUserName, SubjectLogonId, IpAddress, IpPort, RelativeTargetName
FROM mordorTable
WHERE LOWER(Channel) = "security"
    AND EventID = 5145
    AND ShareName LIKE '%C$'
    AND NOT SubjectUserName LIKE '%$'
    AND AccessMask = '0x2'
'''
)
df.show(10,False)

### Analytic IV
Look for non-system accounts SMB connecting (Tree Connect) to an IPC$ Share and administrative shares (i.e C$) and accessing/creating a file with write (0x2) access mask with the same logon session ID.


| Data source | Event Provider | Relationship | Event |
|:------------|:---------------|--------------|-------|
| File | Microsoft-Windows-Security-Auditing | User accessed file share | 5140 |
| File | Microsoft-Windows-Security-Auditing | User accessed File | 5145 |

In [ ]:
df = spark.sql(
'''
SELECT `@timestamp`, Hostname, ShareName, SubjectUserName, d.SubjectLogonId, IpAddress, IpPort, RelativeTargetName
FROM mordorTable d
INNER JOIN (
    SELECT b.SubjectLogonId
    FROM mordorTable b
    INNER JOIN (
        SELECT SubjectLogonId
        FROM mordorTable
        WHERE LOWER(Channel) = "security"
            AND EventID = 5140
            AND ShareName LIKE '%IPC$'
            AND NOT SubjectUserName LIKE '%$'
    ) a
    ON b.SubjectLogonId = a.SubjectLogonId
    WHERE LOWER(b.Channel) = 'security'
        AND b.EventID = 5140
        AND b.ShareName LIKE '%C$'
) c
ON d.SubjectLogonId = c.SubjectLogonId
WHERE LOWER(d.Channel) = 'security'
    AND d.EventID = 5145
    AND d.ShareName LIKE '%C$'
    AND d.AccessMask = '0x2'
'''
)
df.show(10,False)

### Analytic V
Look for files that were accessed over the network with write (0x2) access mask via administrative shares (i.e C$) and that were created by the System process on the target system.


| Data source | Event Provider | Relationship | Event |
|:------------|:---------------|--------------|-------|
| File | Microsoft-Windows-Security-Auditing | User accessed File | 5145 |
| File | Microsoft-Windows-Sysmon/Operational | Process created File | 11 |

In [ ]:
df = spark.sql(
'''
SELECT `@timestamp`, Hostname, ShareName, SubjectUserName, SubjectLogonId, IpAddress, IpPort, RelativeTargetName
FROM mordorTable b
INNER JOIN (
    SELECT LOWER(REVERSE(SPLIT(TargetFilename, '\'))[0]) as TargetFilename
    FROM mordorTable
    WHERE Channel = 'Microsoft-Windows-Sysmon/Operational'
        AND Image = 'System'
        AND EventID = 11
) a
ON LOWER(REVERSE(SPLIT(RelativeTargetName, '\'))[0]) = a.TargetFilename
WHERE LOWER(b.Channel) = 'security'
    AND b.EventID = 5145
    AND b.AccessMask = '0x2'
'''
)
df.show(10,False)

## Known Bypasses


| Idea | Playbook |
|:-----|:---------|

## False Positives
None

## Hunter Notes
* Baseline your environment to identify normal activity. Document all accounts creating files over the network via administrative shares.

## Hunt Output

| Type | Link |
| :----| :----|

## References
* https://docs.microsoft.com/en-us/openspecs/windows_protocols/ms-smb/8341356c-ede3-4e1c-a056-3de91473bde6
* https://docs.microsoft.com/en-us/openspecs/windows_protocols/ms-smb2/e8fb45c1-a03d-44ca-b7ae-47385cfd7997